In [ ]:
class aim_manipulation:
    
    '''
    Class of functions to return a dataframe version of some of the reports available in Bloomberg AIM:
    
    - Cash Ladder
    - Financing Trade Blotter
    - Collateral Positions
    
    These dataframes are the input of the following classes:
    
    - knapsack
    - dashboards
    - repo_trading
    
    Note:
    
    Import the following packages befor running the functions:
    
    import pandas as pd
    import numpy as np
    from datetime import datetime, timedelta
    
    '''
    
    @staticmethod
    def cash_ladder(path, reformat = False):
        
        '''
        Return a dataframe containing the Cash Ladder for all funds included in your AIM account.
        
        path (str): path to the .xlsx file exported from Bloomberg ERA report;
        reformat (boolean): default value is False, if reformat = True the df is formatted with "," for thousands 
        and no decimals.
        
        Note: 
        
        If reformat = True, the resulting dataframe cannot be used as an input for the abovementioned functions.
        The adjustment ".iloc[:110]" removes Bloomberg text disclaimers from the dataframe, but the number of rows depends on
        the funds included in your ERA report.
        
        '''
    
        df = pd.read_excel(path, skiprows = 6).drop(columns = ["Unnamed: 1"]).rename(columns = {"Settle Date Cash Amount" : "Account", 
                                                                                            
                                                                                            "Unnamed: 2" : "Currency"}).iloc[:110].fillna(method = "ffill")            
        
        def format_with_commas(value):
            
            if isinstance(value, (int, float)):
        
                return f'{value:,.0f}'
        
            else:
                                    
                return value
            
        if reformat == True:
            
            return df.applymap(format_with_commas)
        
        else:
            
            return df      
    
    @staticmethod
    def financing_trade_blotter(path):
        
        '''
        Return a dataframe containing the financing trades for all funds.
        
        path (str): path to the .xlsx file exported XFIN > Financing Trade Blotter in Bloomberg AIM.
        
        '''
    
        df = pd.read_excel(path)
    
        for col in df:
        
            if df[col].dtype == "object":
            
                df[col] = df[col].str.lstrip()
            
                df[col] = df[col].str.rstrip()
        
        df = df.drop(0, axis = 0)

        return df
    
    @staticmethod
    def unwind_blotter(path, account, curr, tran_type, term_dt):
        
        '''
        Return a cleaned version of the financing trade blotter obtained from function financing_trade_blotter().
                
        path (str): path to the .xlsx file exported XFIN > Financing Trade Blotter in Bloomberg AIM;
        account (list/str): account name as reported in Bloomberg AIM;
        curr (list): currencies you want to include;
        tran_type (list): transaction types you want to inclued;
        term_dt (str): termination settlement date, in format "%m/%d/%y".
        
        Note:
        
        To view financing trades for all accounts, input account = "all_account".
        This dataframe is used as an input for function "close_repo"; hence, a settlement date must be specified.
        Transaction types are RR, CRR, DO, RP
        - RR: Reverse Repo (You lend the bond), used for funding
        - CRR: Reverse Repo (You lend the bond), used for funding
        - DO: Deposit
        - RP: Repo (You borrow the bond), used for shorting       
                
        '''
        
        df = aim_manipulation.financing_trade_blotter(path)
        
        day_count = {"EUR" : 360,
                     "USD" : 360,
                     "AUD" : 365}
        
        if account != "all_account":
            
            df = df[df["Account"].isin(account)]
        
        df = df[["Account", "Tran Type", "Broker", "Collateral", "Identifier", "Quantity", "Curr", "Money", "Rate", "SetDt"]]
        
        df["SetDt"] = pd.to_datetime(df["SetDt"], format = "%m/%d/%y")
               
        df["TermDt"] = pd.to_datetime(datetime.strptime(term_dt, '%d/%m/%Y'))
        
        df = df[(df["Curr"].isin(curr)) & (df["Tran Type"].isin(tran_type))].reset_index(drop = True)
        
        df["DayCount"] = df["Curr"].map(day_count)
        
        df["Daily Interest"] = df["Money"] * ((df["Rate"] / 365) / 100)

        df["Accr Interest"] = df["Money"] * (df["Rate"] * ((df["TermDt"] - df["SetDt"]).dt.days / 365) / 100)
        
        df["Term Amount"] = df["Money"] + df["Accr Interest"]
                
        return df
    
    @staticmethod
    def collateral_positions(path):
        
        '''
        Return a dataframe containing collateral positions for all funds.
        
        path (str): path to the .xlsx file exported XFIN > Collateral Positions in Bloomberg AIM;
        
        Note:
        
        This function is based on headers of the Collateral Positions blotter displayed in the following order:
        Account, Special, Security, Curr, ISIN, Pos, Open RP, Open RR, "t+0", "t+1", "t+2", "t+3", "t+4", Mty, Px, 
        Series, Sect, Rank, TermRP, TermRR. 
        
        '''
        
        df = pd.read_excel(path, skiprows = [1])
        
        df.rename(columns = {
            
            df.columns[8] : "T+0",
            
            df.columns[9] : "T+1",
            
            df.columns[10] : "T+2",
            
            df.columns[11] : "T+3",
            
            df.columns[12] : "T+4",
            
        },
            
            inplace = True
            
        )
        
        return df